In [311]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from scipy.stats.mstats import winsorize

### **Panel A: Emission variables**

In [312]:
# load data as pandas data frames
dataset_raw = pd.read_csv('dataset_raw.csv')

# store emission variables
emission_variables =  pd.DataFrame()

In [313]:
dataset_sorted = dataset_raw.sort_values(['gvkey', 'fyear'])

mask = (dataset_sorted['Scope_1'] > 0) & (dataset_sorted['Scope_2'] > 0)
dataset_sorted['Scope_1'] = dataset_sorted['Scope_1'][mask]
dataset_sorted['Scope_2'] = dataset_sorted['Scope_2'][mask]

#### Log(total Emission)

In [314]:
emission_variables['total_emission_scope1'] = np.log(dataset_sorted['Scope_1'])
emission_variables['total_emission_scope2'] = np.log(dataset_sorted['Scope_2'])

In [315]:
# store 'gvkey' and 'fyear' from the sorted dataset in the emission_variables dictionary
emission_variables['gvkey'] = dataset_sorted['gvkey']
emission_variables['fyear'] = dataset_sorted['fyear']

#### Emission Growth Rate

In [316]:
emission_variables['emission_growth_scope1'] = dataset_sorted.groupby('gvkey')['Scope_1'].pct_change()
emission_variables['emission_growth_scope2'] = dataset_sorted.groupby('gvkey')['Scope_2'].pct_change()

C:\Users\18906\AppData\Local\Temp\ipykernel_13520\1937878722.py:1: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  emission_variables['emission_growth_scope1'] = dataset_sorted.groupby('gvkey')['Scope_1'].pct_change()
C:\Users\18906\AppData\Local\Temp\ipykernel_13520\1937878722.py:2: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  emission_variables['emission_growth_scope2'] = dataset_sorted.groupby('gvkey')['Scope_2'].pct_change()


In [317]:
growth_rates1 = emission_variables['emission_growth_scope1']
growth_rates2 = emission_variables['emission_growth_scope2']

# inf, nan
growth_rates1.replace([np.inf, -np.inf], np.nan, inplace=True)
growth_rates1.dropna(inplace=True)
growth_rates2.replace([np.inf, -np.inf], np.nan, inplace=True)
growth_rates2.dropna(inplace=True)

# winsorize
winsorized_growth_rates1 = winsorize(growth_rates1, limits=[0.025, 0.025])
winsorized_growth_rates2 = winsorize(growth_rates2, limits=[0.025, 0.025])

In [318]:
# replace the emission growth by winsorized data
emission_variables['emission_growth_scope1'] = pd.DataFrame(winsorized_growth_rates1)
emission_variables['emission_growth_scope2'] = pd.DataFrame(winsorized_growth_rates2)

### Carbon Intensity

In [319]:
emission_variables['carbon_intensity_scope1'] = (dataset_sorted['Scope_1'] / (dataset_sorted['sale']*100)) 
emission_variables['carbon_intensity_scope2'] = (dataset_sorted['Scope_2'] / (dataset_sorted['sale']*100)) 

# inf, nan #
emission_variables['carbon_intensity_scope1'].replace([np.inf, -np.inf], np.nan, inplace=True)
emission_variables['carbon_intensity_scope1'].dropna(inplace=True)
emission_variables['carbon_intensity_scope2'].replace([np.inf, -np.inf], np.nan, inplace=True)
emission_variables['carbon_intensity_scope2'].dropna(inplace=True)

# winsorize
emission_variables['carbon_intensity_scope1'] = winsorize(emission_variables['carbon_intensity_scope1'], limits=[0.025, 0.025])
emission_variables['carbon_intensity_scope2'] = winsorize(emission_variables['carbon_intensity_scope2'], limits=[0.025, 0.025])

C:\Users\18906\AppData\Local\Temp\ipykernel_13520\3470311249.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  emission_variables['carbon_intensity_scope1'].replace([np.inf, -np.inf], np.nan, inplace=True)
C:\Users\18906\AppData\Local\Temp\ipykernel_13520\3470311249.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are set

### Check emission data

In [320]:
emission_variables.describe()

c:\Users\18906\Python\Lib\site-packages\numpy\lib\function_base.py:4823: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(
c:\Users\18906\Python\Lib\site-packages\numpy\lib\function_base.py:4823: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(
c:\Users\18906\Python\Lib\site-packages\numpy\lib\function_base.py:4823: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(
c:\Users\18906\Python\Lib\site-packages\numpy\lib\function_base.py:4823: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(
c:\Users\18906\Python\Lib\site-packages\numpy\lib\function_base.py:4823: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(
c:\Users\18906\Python\Lib\site-packages\numpy\lib\function_base.py:4823: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition

,total_emission_scope1,total_emission_scope2,gvkey,fyear,emission_growth_scope1,emission_growth_scope2,carbon_intensity_scope1,carbon_intensity_scope2
count,15987.000000,15987.000000,15999.000000,15999.000000,13365.000000,13365.000000,15999.000000,15999.000000
mean,9.880214,10.095786,55747.186137,2016.570911,0.056679,0.068529,1.163097,0.303205
std,2.978628,2.464885,64393.751756,2.857792,0.295652,0.292209,3.586708,0.370308
min,-1.645065,-0.079043,1004.000000,2010.000000,-0.587399,-0.510633,0.003999,0.008185
25%,7.884177,8.627100,9815.000000,2015.000000,-0.070276,-0.074187,0.035725,0.073925
50%,9.816265,10.300611,24430.000000,2017.000000,0.023288,0.024767,0.130434,0.168343
75%,11.627291,11.748831,66731.000000,2019.000000,0.133633,0.155335,0.294071,0.403065
max,18.804073,16.920606,328795.000000,2020.000000,1.124937,1.138111,19.735829,1.780249


### **Paneml B: Cross-sectional return variables**

In [372]:
# read data
dataset_raw = pd.read_csv('dataset_raw.csv')
dataset_sorted = dataset_raw.sort_values(['gvkey', 'fyear'])

In [322]:
# store the computed variables using only dataset_raw in a new pd dataframe
return_variables_raw = pd.DataFrame()
return_variables_raw['gvkey'] = dataset_sorted['gvkey']
return_variables_raw['fyear'] = dataset_sorted['fyear']

#### LOGSIZE: natural logarithm of market capitalization (in $ million)

In [323]:
# market capitalization = stock price * # of outstanding shares
return_variables_raw['LOGSIZE'] = np.log(dataset_raw['csho']*dataset_raw['Price_Close'])

mean_logsize = return_variables_raw['LOGSIZE'].mean()
median_logsize = return_variables_raw['LOGSIZE'].median()
std_logsize = return_variables_raw['LOGSIZE'].std()

print(f"Mean LOGSIZE: {mean_logsize}")
print(f"Median LOGSIZE: {median_logsize}")
print(f"Standard Deviation of LOGSIZE: {std_logsize}")

Mean LOGSIZE: 8.007593586954304
Median LOGSIZE: 8.002279656850122
Standard Deviation of LOGSIZE: 1.8085235360999352


#### B/M (winsorized at 2.5%): book value of equity divided by market value of equity

In [324]:
dataset_raw['Equity_Book_Value'] = dataset_raw['at']-dataset_raw['lt'] # compute book value
dataset_raw['Equity_Market_Value'] = dataset_raw['csho']*dataset_raw['Price_Close'] # compute market value

# compute B/M and winsorized at 2.5%
return_variables_raw['B/M'] = winsorize(dataset_raw['Equity_Book_Value']/dataset_raw['Equity_Market_Value'], limits=[0.025, 0.025]) 

In [325]:
mean_bm = np.mean(return_variables_raw['B/M'])
median_bm = np.median(return_variables_raw['B/M'])
std_dev_bm = np.std(return_variables_raw['B/M'])

print(f"Mean B/M: {mean_bm}")
print(f"Median B/M: {median_bm}")
print(f"Standard Deviation of B/M: {std_dev_bm}")

Mean B/M: 0.5850595900650135
Median B/M: 0.4447816521232229
Standard Deviation of B/M: 0.5424168173950612


#### LEVERAGE (winsorized at 2.5%): book value of leverage, i.e. book value of debt divided by the book value of assets

In [326]:
dataset_raw['Debt_Book_Value'] = dataset_raw['dlc'] + dataset_raw['dltt']
dataset_raw['Assets_Book_Value'] = dataset_raw['at']
# no DPACT -- Depreciation, Depletion and Amortization (Accumulated) (DPACT) recorded in dataset_raw
return_variables_raw['LEVERAGE'] = winsorize(dataset_raw['Debt_Book_Value']/dataset_raw['Assets_Book_Value'], limits=[0.025,0.025])

In [327]:
mean_leverage = np.mean(return_variables_raw['LEVERAGE'])
median_leverage = np.median(return_variables_raw['LEVERAGE'])
std_dev_leverage = np.std(return_variables_raw['LEVERAGE'])

print(f"Mean LEVERAGE: {mean_leverage}")
print(f"Median LEVERAGE: {median_leverage}")
print(f"Standard Deviation of LEVERAGE: {std_dev_leverage}")

Mean LEVERAGE: 0.27797261602552453
Median LEVERAGE: 0.2547043776419151
Standard Deviation of LEVERAGE: 0.2137803433861781


#### INVEST/A (winsorized at 2.5%): CAPEX divided by book value of assets

In [328]:
return_variables_raw['INVEST/A'] = dataset_raw['capx']/dataset_raw['Assets_Book_Value']

# winsorize
return_variables_raw['INVEST/A'] = winsorize(return_variables_raw['INVEST/A'], limits=[0.025 ,0.025])

In [329]:
mean_investa = np.mean(return_variables_raw['INVEST/A'])
median_investa = np.median(return_variables_raw['INVEST/A'])
std_dev_investa = np.std(return_variables_raw['INVEST/A'])

print(f"Mean INVEST/A: {mean_investa}")
print(f"Median INVEST/A: {median_investa}")
print(f"Standard Deviation of INVEST/A: {std_dev_investa}")

Mean INVEST/A: 0.0331340162202419
Median INVEST/A: 0.021296996330072926
Standard Deviation of INVEST/A: 0.0373185785949221


#### ROE (winsorized at 2.5%, in%): return on equity, i.e. Net Income / Shareholder's Equity

In [330]:
return_variables_raw['ROE'] = (dataset_raw['ni']/dataset_raw['seq'])*100

# winsorize
return_variables_raw['ROE'] = winsorize(return_variables_raw['ROE'], limits=[0.025,0.025])

In [331]:
mean_roe = np.mean(return_variables_raw['ROE'])
median_roe = np.median(return_variables_raw['ROE'])
std_dev_roe = np.std(return_variables_raw['ROE'])

print(f"Mean ROE: {mean_roe}")
print(f"Median ROE: {median_roe}")
print(f"Standard Deviation of ROE: {std_dev_roe}")

Mean ROE: 6.7132544174558175
Median ROE: 9.511803804721522
Standard Deviation of ROE: 31.57610126599628


#### <font color= gray>HHI: Herfindahl index of the business segments of a company with weights proportional to revenues

1. Identify the Business Segments and Their Revenues

2. Calculate the Market Share of Each Segment

3. Square Each Segment's Market Share: For each business segment, square its market share. If you're using percentages, make sure to convert them into fractions first (e.g., 20% becomes 0.20). Squaring the market share amplifies the weight of larger segments.

4. Sum the Squared Market Shares

5. Interpret the HHI: The HHI can range from close to 0 to 10,000 (if using percentages, it ranges from 0 to 1). A lower HHI indicates a more diversified company with no single business segment dominating its revenue, while a higher HHI suggests a higher concentration in one or a few segments.

In [332]:
# revt: Total Revenue
# no segemental revenue in datase_raw
# lack of data to compute

#### LOGPPE: natural logarithm of plant, property & equipment (in $ million)

In [333]:
# avoid nan and 0
ppegt_filtered = dataset_sorted['ppegt'].dropna()
ppegt_filtered = ppegt_filtered[ppegt_filtered != 0]

return_variables_raw['LOGPPE'] = np.log(ppegt_filtered)

In [334]:
mean_loppe = np.mean(return_variables_raw['LOGPPE'])
median_loppe = return_variables_raw['LOGPPE'].median()
std_dev_loppe = np.std(return_variables_raw['LOGPPE'])

print(f"Mean loppe: {mean_loppe}")
print(f"Median loppe: {median_loppe}")
print(f"Standard Deviation of loppe: {std_dev_loppe}")

Mean loppe: 6.747088411589938
Median loppe: 6.825568554954745
Standard Deviation of loppe: 2.2368158701703473


#### SALESGR (winsorized at 0.5%): sales growth rate; dollar change in annual firm revenues normalized by last month’s market capitalization;

In [335]:
# Calculate the correct market capitalization
dataset_raw['mkvalt'] = dataset_raw['csho'] * dataset_raw['Price_Close']

# Drop rows with NaN values in 'revt' and 'mkvalt' columns
dataset_clean = dataset_raw.dropna(subset=['revt', 'mkvalt'])

# Sort by 'gvkey' and 'fyear' to ensure proper time series alignment
dataset_sorted = dataset_clean.sort_values(['gvkey', 'fyear'])

# Shift 'revt' values within each 'gvkey' group
def shift_revenue(group):
    group['revt_t-1'] = group['revt'].shift(1)
    return group

revenue_shifted = dataset_sorted.groupby('gvkey').apply(shift_revenue).reset_index(drop=True)

# Calculate the dollar change in annual revenues
revenue_shifted['REVT_change'] = revenue_shifted['revt'] - revenue_shifted['revt_t-1']

# Shift 'mkvalt' values to get the market capitalization from the previous month
dataset_sorted['mkvalt_t-1'] = dataset_sorted.groupby('gvkey')['mkvalt'].shift(1)

# Merge revenue_shifted with dataset_sorted to align the market capitalizations
merged_data_salesgr = pd.merge(revenue_shifted, dataset_sorted[['gvkey', 'fyear', 'mkvalt_t-1']], on=['gvkey', 'fyear'])

# Normalize REVT_change by the previous month's market capitalization
merged_data_salesgr['SALESGR'] = merged_data_salesgr['REVT_change'] / merged_data_salesgr['mkvalt_t-1']

# Replace infinite values with NaN
merged_data_salesgr['SALESGR'] = merged_data_salesgr['SALESGR'].replace([np.inf, -np.inf], np.nan)

# Drop NaN values for winsorization
SALESGR_cleaned = merged_data_salesgr['SALESGR'].dropna()

# Winsorize the 'SALESGR' column, removing the top and bottom 0.5%
SALESGR_winsorized = winsorize(SALESGR_cleaned, limits=[0.005, 0.005])

# Update the non-NaN 'SALESGR' values with the winsorized values
merged_data_salesgr.loc[SALESGR_cleaned.index, 'SALESGR'] = SALESGR_winsorized

# Drop rows where 'SALESGR' is NaN
cleaned_data_salesgr = merged_data_salesgr.dropna(subset=['SALESGR'])

# Extract the required columns
salesgr = cleaned_data_salesgr[['SALESGR', 'gvkey', 'fyear']]

# Merge salesgr with return_variables_raw
return_variables_raw = pd.merge(return_variables_raw, salesgr, on=['gvkey', 'fyear'], how='left')

C:\Users\18906\AppData\Local\Temp\ipykernel_13520\477207372.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  revenue_shifted = dataset_sorted.groupby('gvkey').apply(shift_revenue).reset_index(drop=True)


In [336]:
mean_salesgr = np.mean(return_variables_raw['SALESGR'])
median_salesgr = return_variables_raw['SALESGR'].median()
std_dev_salesgr = np.std(return_variables_raw['SALESGR'])

print(f"Mean salesgr: {mean_salesgr}")
print(f"Median salesgr: {median_salesgr}")
print(f"Standard Deviation of salesgr: {std_dev_salesgr}")

Mean salesgr: 0.021343175236726464
Median salesgr: 0.020862000586642178
Standard Deviation of salesgr: 0.31204093140103867


In [337]:
return_variables_raw.to_csv('./return_variables_raw_s1.csv', index=True)

In [338]:
return_variables_raw = return_variables_raw.dropna()
return_variables_raw.describe()

,gvkey,fyear,LOGSIZE,B/M,LEVERAGE,INVEST/A,ROE,LOGPPE,SALESGR
count,11399.000000,11399.000000,11399.000000,11399.000000,11399.000000,11399.000000,11399.000000,11399.000000,11399.000000
mean,53924.081762,2016.822090,8.221267,0.589930,0.277220,0.038886,7.429493,6.976566,0.023127
std,64420.750701,2.699119,1.909072,0.547446,0.214631,0.037162,33.364842,2.184767,0.327308
min,1004.000000,2010.000000,-4.612195,-0.040954,0.000000,0.000000,-109.224422,-2.465104,-2.260408
25%,8151.000000,2015.000000,7.068393,0.228919,0.091842,0.013196,1.713317,5.600710,-0.011221
50%,22182.000000,2017.000000,8.255643,0.449925,0.252536,0.027083,10.708132,7.035269,0.022269
75%,66393.000000,2019.000000,9.437186,0.775888,0.411960,0.051824,19.534730,8.461259,0.081140
max,328795.000000,2020.000000,20.534197,2.663034,0.814584,0.163615,91.432361,13.144907,1.647431


***

In [339]:
# read data
eps_marketLeverage = pd.read_csv('eps_marketLeverage.csv') 

In [340]:
# filter the data by gvkey
companies_emission_data = dataset_raw['gvkey'].unique()
dataset_sorted = dataset_raw.sort_values(['gvkey', 'fyear'])

eps_marketLeverage_f = eps_marketLeverage[eps_marketLeverage['gvkey'].isin(companies_emission_data)]
eps_marketLeverage_f = eps_marketLeverage_f.sort_values(by = ['gvkey', 'fyear'])

#### EPSGR (winsorized at 0.5%): Earnings Per Share Growth Rate; dollar change in annual earnings per share, normalized by the firm’s equity price

In [341]:
# store the computed variables using only dataset_raw in a new pd dataframe
return_variables_eps = pd.DataFrame()
return_variables_eps['gvkey'] = eps_marketLeverage['gvkey']
return_variables_eps['fyear'] = eps_marketLeverage['fyear']

In [342]:
return_variables_eps.to_csv('./return_variables_eps_s.csv', index=True)

In [343]:
###### Create a copy of eps_marketLeverage_f
eps_marketLeverage_copy = eps_marketLeverage_f.copy()

###### Drop rows with NaN values in the 'EPS' column
eps_marketLeverage_copy = eps_marketLeverage_copy.dropna(subset=['EPS'])

###### Sort by 'gvkey' and 'fyear' to ensure proper time series alignment
eps_marketLeverage_sorted = eps_marketLeverage_copy.sort_values(['gvkey', 'fyear'])

###### Shift EPS values within each 'gvkey' group
def shift_if_consecutive(group):
    group['EPS_t-1'] = group['EPS'].shift(1)
    return group

eps_shifted = eps_marketLeverage_sorted.groupby('gvkey').apply(shift_if_consecutive).reset_index(drop=True)

###### Calculate the dollar change in annual earnings per share
eps_shifted['EPS_change'] = eps_shifted['EPS'] - eps_shifted['EPS_t-1']



###### Assuming dataset_sorted has a 'Price_Close' column with equity prices
dataset_sorted = dataset_sorted.sort_values(['gvkey', 'fyear'])
dataset_sorted['Price_Close_t-1'] = dataset_sorted.groupby('gvkey')['Price_Close'].shift(1)

###### Merge eps_shifted with dataset_sorted to align the equity prices
merged_data_eps = pd.merge(eps_shifted, dataset_sorted[['gvkey', 'fyear', 'Price_Close_t-1']], on=['gvkey', 'fyear'])

###### Normalize EPS_change by the firm's equity price
merged_data_eps['EPSGR'] = merged_data_eps['EPS_change'] / merged_data_eps['Price_Close_t-1']

###### Replace infinite values with NaN
merged_data_eps['EPSGR'] = merged_data_eps['EPSGR'].replace([np.inf, -np.inf], np.nan)

###### Drop NaN values for winsorization
EPSGR_cleaned = merged_data_eps['EPSGR'].dropna()

###### Winsorize the 'EPSGR' column, removing the top and bottom 0.5%
EPSGR_winsorized = winsorize(EPSGR_cleaned, limits=[0.005, 0.005])

###### Update the non-NaN 'EPSGR' values with the winsorized values
merged_data_eps.loc[EPSGR_cleaned.index, 'EPSGR'] = EPSGR_winsorized

###### Drop rows where 'EPSGR' is NaN
cleaned_data_eps = merged_data_eps.dropna(subset=['EPSGR'])



return_variables_eps = cleaned_data_eps[['EPSGR', 'gvkey', 'fyear']]

C:\Users\18906\AppData\Local\Temp\ipykernel_13520\3732974984.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  eps_shifted = eps_marketLeverage_sorted.groupby('gvkey').apply(shift_if_consecutive).reset_index(drop=True)


In [344]:
return_variables_eps.to_csv('./return_variables_eps_s1.csv', index=True)

In [345]:
mean_epsgr = np.mean(return_variables_eps['EPSGR'])
median_epsgr = return_variables_eps['EPSGR'].median()
std_dev_epsgr = np.std(return_variables_eps['EPSGR'])

print(f"Mean EPSGR: {mean_epsgr}")
print(f"Median EPSGR: {median_epsgr}")
print(f"Standard Deviation of EPSGR: {std_dev_epsgr}")

Mean EPSGR: 0.005474900523590049
Median EPSGR: 0.003826981258366801
Standard Deviation of EPSGR: 0.22759708521046187


***

In [346]:
# read data
weekly_equity_returns = pd.read_csv('weekly_equity_returns.csv')

In [347]:
# filter the data by gvkey
companies_emission_data = dataset_raw['gvkey'].unique()

weekly_equity_returns_f = weekly_equity_returns[weekly_equity_returns['gvkey'].isin(companies_emission_data)]
weekly_equity_returns_f = weekly_equity_returns_f.sort_values(by = ['gvkey', 'eow'])

#### RET(%): monthly stock return

In [373]:
return_variables_ret = pd.DataFrame()

In [374]:
weekly_equity_returns_f['eow'] = pd.to_datetime(weekly_equity_returns_f['eow'])
weekly_equity_returns_f['eow_ret'] = pd.to_datetime(weekly_equity_returns_f['eow'])
weekly_equity_returns_f['eow_mom'] = pd.to_datetime(weekly_equity_returns_f['eow']).copy()
weekly_equity_returns_f['eow_volat'] = pd.to_datetime(weekly_equity_returns_f['eow']).copy()
weekly_equity_returns_f.set_index('eow', inplace=True)

# Reweekly_equity_returns to get the last entry of each month
monthly_data = weekly_equity_returns_f.groupby('gvkey').resample('M').last()
return_variables_ret = monthly_data[['gvkey']].copy

monthly_prices = weekly_equity_returns_f.groupby(['gvkey', pd.Grouper(freq='M')])['Close_Price'].last()

monthly_returns = monthly_prices.groupby(level=0).pct_change() * 100
monthly_returns = monthly_returns.reset_index()

# month column
monthly_returns['month'] = monthly_returns['eow'].dt.to_period('M')

# rename
monthly_returns.rename(columns={'Close_Price': 'RET'}, inplace=True)
monthly_returns.dropna(inplace=True)

C:\Users\18906\AppData\Local\Temp\ipykernel_13520\1391781365.py:8: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_data = weekly_equity_returns_f.groupby('gvkey').resample('M').last()
C:\Users\18906\AppData\Local\Temp\ipykernel_13520\1391781365.py:11: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_prices = weekly_equity_returns_f.groupby(['gvkey', pd.Grouper(freq='M')])['Close_Price'].last()


In [350]:
monthly_returns = monthly_returns[monthly_returns['RET'] <= 100]
return_variables_ret = monthly_returns.copy()
return_variables_ret = return_variables_ret.dropna(subset=['RET'])

In [351]:
return_variables_ret.drop(columns=['eow'])

# Split 'month' column into 'fyear' and 'month'
return_variables_ret['month'] = return_variables_ret['month'].astype(str)
return_variables_ret['fyear'] = return_variables_ret['month'].str[:4].astype(int)
return_variables_ret['month'] = return_variables_ret['month'].str[5:].astype(int)

# Rearrange columns to have 'gvkey', 'fyear', 'month', and 'RET'
return_variables_ret = return_variables_ret[['gvkey', 'fyear', 'month', 'RET']]

In [352]:
return_variables_ret.describe()

,gvkey,fyear,month,RET
count,339930.000000,339930.000000,339930.000000,339930.000000
mean,61166.521357,2016.546489,6.557141,1.007106
std,66146.159969,3.667320,3.438243,12.474702
min,1004.000000,2010.000000,1.000000,-99.989583
25%,11511.000000,2013.000000,4.000000,-5.134318
50%,27119.000000,2017.000000,7.000000,0.738663
75%,119574.000000,2020.000000,10.000000,6.606061
max,328795.000000,2022.000000,12.000000,100.000000


In [353]:
return_variables_ret.to_csv('./return_variables_ret_s.csv', index=True)

In [354]:
# alternative annual return
total_return = pd.read_csv('total_returns.csv')

ret1 = total_return[['Total Return', 'gvkey', 'fyear']]
ret1['RET1'] = ret1['Total Return']
ret1 = ret1.drop('Total Return', axis=1)

C:\Users\18906\AppData\Local\Temp\ipykernel_13520\3990806797.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ret1['RET1'] = ret1['Total Return']


In [355]:
ret1['RET1'] = winsorize(ret1['RET1'], limits=[0.025,0.025])

In [356]:
ret1['RET1'].describe()

c:\Users\18906\Python\Lib\site-packages\numpy\lib\function_base.py:4823: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(


count    27052.000000
mean        15.892362
std         39.831668
min        -57.398563
25%         -9.274308
50%         11.931753
75%         34.955763
max        135.390428
Name: RET1, dtype: float64

#### MOM (winsorized at 0.5%): cumulative stock return over the one-year period

In [357]:
return_variables_mom = pd.DataFrame()

In [358]:
# recall we already have 'eow_copy' in pd datetime, add months and years as columns
# the former 'eow_copy' cannot be used as nan vals when computing ret were dropped
weekly_equity_returns_f['week'] = pd.to_datetime(weekly_equity_returns_f['eow_mom'])
weekly_equity_returns_f['month'] = weekly_equity_returns_f['week'].dt.month
weekly_equity_returns_f['year'] = weekly_equity_returns_f['week'].dt.year

# compute the weekly close price for every company 
weekly_prices = weekly_equity_returns_f.groupby(['gvkey', 'year', 'week'])['Close_Price'].last().reset_index()
weekly_prices.sort_values(by=['gvkey', 'year', 'week'], inplace=True)

# construct yearly window (12 months)
weekly_prices['Close_Price_1year_ago'] = weekly_prices.groupby('gvkey')['Close_Price'].shift(52)

# Ensure no NaN values in 'Close_Price'
weekly_prices.dropna(subset=['Close_Price', 'Close_Price_1year_ago'], inplace=True)

# compute MOM
weekly_prices['MOM'] = weekly_prices['Close_Price'] / weekly_prices['Close_Price_1year_ago'] - 1

# store the result in return_variables_mom
return_variables_mom = weekly_prices[['gvkey', 'week', 'MOM']]

# winsorize
return_variables_mom['MOM'] = winsorize(return_variables_mom['MOM'], limits=[0.005, 0.005])

C:\Users\18906\AppData\Local\Temp\ipykernel_13520\2808210754.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return_variables_mom['MOM'] = winsorize(return_variables_mom['MOM'], limits=[0.005, 0.005])


In [359]:
# Ensure the 'week' column is in datetime format
return_variables_mom['week'] = pd.to_datetime(return_variables_mom['week'])

# Extract year and month from 'week'
return_variables_mom['fyear'] = return_variables_mom['week'].dt.year
return_variables_mom['month'] = return_variables_mom['week'].dt.month

# Aggregate the 'MOM' values by 'gvkey', 'fyear', and 'month'
return_variables_mom = return_variables_mom.groupby(['gvkey', 'fyear', 'month'])['MOM'].mean().reset_index()

C:\Users\18906\AppData\Local\Temp\ipykernel_13520\958719108.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return_variables_mom['week'] = pd.to_datetime(return_variables_mom['week'])
C:\Users\18906\AppData\Local\Temp\ipykernel_13520\958719108.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return_variables_mom['fyear'] = return_variables_mom['week'].dt.year
C:\Users\18906\AppData\Local\Temp\ipykernel_13520\958719108.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [360]:
# compute mean, median, std
mean_mom = return_variables_mom['MOM'].mean()
median_mom = return_variables_mom['MOM'].median()
std_dev_mom = return_variables_mom['MOM'].std()

print(f"Mean MOM: {mean_mom}")
print(f"Median MOM: {median_mom}")
print(f"Standard Deviation of MOM: {std_dev_mom}")

Mean MOM: 0.14551378797374853
Median MOM: 0.07941700004975033
Standard Deviation of MOM: 0.5028003259501682


In [361]:
return_variables_mom.to_csv('./return_variables_mom_s.csv', index=True)

#### Beta: CAMP beta calculated over the on year period

In [362]:
betas = pd.read_csv('betas.csv')
betas = betas.dropna()

In [363]:
betas['BETA'] = winsorize(betas['beta'], limits=[0.005, 0.005])

In [364]:
betas = betas.drop('beta', axis=1)
betas = betas.drop('winsorized_beta', axis=1)

# Ensure the 'eow' column is in datetime format
betas['eow'] = pd.to_datetime(betas['eow'])

# Extract year and month from 'eow'
betas['fyear'] = betas['eow'].dt.year
betas['month'] = betas['eow'].dt.month

# Group by 'gvkey', 'fyear', and 'month' to calculate the average BETA
betas = betas.groupby(['gvkey', 'fyear', 'month'])['BETA'].mean().reset_index()

In [365]:
betas['BETA'].describe()

count    318352.000000
mean          1.112173
std           0.718387
min          -1.044560
25%           0.669873
50%           1.046832
75%           1.474245
max           4.147888
Name: BETA, dtype: float64

In [366]:
betas.to_csv('./betas_s.csv', index=True)

#### VOLAT (winsorized at 0.5%): monthly stock return volatility calculated over the one year period

In [367]:
return_variables_volat = pd.DataFrame()

In [368]:
weekly_equity_returns_f['eow'] = pd.to_datetime(weekly_equity_returns_f['eow_volat'])

###### Extract year and month from 'eow'
weekly_equity_returns_f['year'] = weekly_equity_returns_f['eow'].dt.year
weekly_equity_returns_f['month'] = weekly_equity_returns_f['eow'].dt.month

###### Group by 'gvkey', 'year', and 'month' to get the monthly closing price
monthly_prices = weekly_equity_returns_f.groupby(['gvkey', 'year', 'month'])['Close_Price'].last().reset_index()

###### Shift 'Close_Price' within each 'gvkey' group to get the previous month's closing price
monthly_prices['Close_Price_t-1'] = monthly_prices.groupby('gvkey')['Close_Price'].shift(1)

###### Calculate the monthly return
monthly_prices['RET'] = (monthly_prices['Close_Price'] - monthly_prices['Close_Price_t-1']) / monthly_prices['Close_Price_t-1']

###### Filter out returns greater than 100%
monthly_prices = monthly_prices[monthly_prices['RET'] <= 1.0]

###### Drop rows with NaN values in 'RET'
monthly_prices = monthly_prices.dropna(subset=['RET'])

###### Calculate the rolling 12-month standard deviation of returns (volatility)
monthly_prices['VOLAT'] = monthly_prices.groupby('gvkey')['RET'].rolling(window=12, min_periods=12).std().reset_index(level=0, drop=True)

###### Drop rows with NaN values in 'VOLAT'
monthly_prices = monthly_prices.dropna(subset=['VOLAT'])

monthly_prices['VOLAT'] = winsorize(monthly_prices['VOLAT'], limits=[0.005,0.005])

###### Create the return_variables_volat DataFrame with the required columns
return_variables_volat = monthly_prices[['gvkey', 'year', 'month', 'VOLAT']]

###### Rename 'year' to 'fyear' to match the desired output
return_variables_volat.rename(columns={'year': 'fyear'}, inplace=True)

C:\Users\18906\AppData\Local\Temp\ipykernel_13520\1723486817.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return_variables_volat.rename(columns={'year': 'fyear'}, inplace=True)


In [369]:
return_variables_volat.describe()

,gvkey,fyear,month,VOLAT
count,311505.000000,311505.000000,311505.000000,311505.000000
mean,61284.341112,2016.966225,6.533622,0.106601
std,66250.276074,3.413744,3.451145,0.060118
min,1004.000000,2011.000000,1.000000,0.025872
25%,11343.000000,2014.000000,4.000000,0.064066
50%,26590.000000,2017.000000,7.000000,0.091142
75%,120093.000000,2020.000000,10.000000,0.132434
max,328795.000000,2022.000000,12.000000,0.356740


In [370]:
return_variables_volat.to_csv('./return_variables_volat_s1.csv', index=True)

***
### Cross-sectional Return Variables Summary Statistics

In [371]:
dataframes = {
    'return_variables': {'RET(%)': return_variables_ret['RET']},
    'return_variables_mom': {'MOM': return_variables_mom['MOM']},
    'betas': {'Beta': betas['BETA']},
    'return_variables_volat': {'VOLAT': return_variables_volat['VOLAT']},
    'return_variables_raw': {'LOGSIZE': return_variables_raw['LOGSIZE'], 'B/M': return_variables_raw['B/M'], 'LEVERAGE': return_variables_raw['LEVERAGE'], 'INVEST/A': return_variables_raw['INVEST/A'], 'ROE': return_variables_raw['ROE'], 'LOGPPE': return_variables_raw['LOGPPE'], 'SALESGR': return_variables_raw['SALESGR']},
    'return_variables_eps': {'EPSGR': return_variables_eps['EPSGR']}
}

summary_stats = pd.DataFrame(index=['mean', 'median', 'std. dev.'])


for df_name, variables in dataframes.items():
    for var_name, var_data in variables.items():
        summary_stats[var_name] = [var_data.mean(), var_data.median(), var_data.std()]

summary_stats = summary_stats.T

print(summary_stats)

              mean     median  std. dev.
RET(%)    1.007106   0.738663  12.474702
MOM       0.145514   0.079417   0.502800
Beta      1.112173   1.046832   0.718387
VOLAT     0.106601   0.091142   0.060118
LOGSIZE   8.221267   8.255643   1.909072
B/M       0.589930   0.449925   0.547446
LEVERAGE  0.277220   0.252536   0.214631
INVEST/A  0.038886   0.027083   0.037162
ROE       7.429493  10.708132  33.364842
LOGPPE    6.976566   7.035269   2.184767
SALESGR   0.023127   0.022269   0.327308
EPSGR     0.005475   0.003827   0.227608
